As an introduction to tensorflow, let's use tensorflow to classify MNIST numerals. However, we will not be using keras -- while keras simplifies constructing basic neural networks, it decreases flexibility that we may need in the future (e.g., when we try to model optical processes). I will try keep this introduction as simple as possible without including too many extra features that, while likely to be useful, may distract you from getting something running for the first time. Once we get the basics down, in the future we can revisit some of the helpful tools, such as graph visualization and tensorboard.

# import data

In [1]:
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt

# load MNIST dataset:
(X_train, y_train), (X_test, y_test) = tf.keras.datasets.mnist.load_data()

# verify that the shapes are correct:
print(X_train.shape)
print(y_train.shape)
print(X_test.shape)
print(y_test.shape)

# cast as a float32; in general, you will work with float32 inputs:
X_train = X_train.astype(np.float32)
X_test = X_test.astype(np.float32)

# add a channels dimension:
X_train = X_train[..., None]
X_test = X_test[..., None]

D:\Anaconda3\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


(60000, 28, 28)
(60000,)
(10000, 28, 28)
(10000,)


# symbolically generate a batch of images and labels

In [2]:
# this can be either X_train/y_train or X_test/y_test, so we make a placeholder that we can feed into:
X_train_or_test = tf.placeholder(tf.float32, [None, 28, 28, 1], name='input_image')
y_train_or_test = tf.placeholder(tf.int32, [None], name='image_label')
batch_size = 32

# create a tf dataset, from which we can generate batches
dataset = tf.data.Dataset.from_tensor_slices((X_train_or_test, y_train_or_test))
dataset = dataset.batch(batch_size)
batch_generator = dataset.make_initializable_iterator()
X_batch, y_batch = batch_generator.get_next()  # batches symbolically generated

# given a symbolic batch, symbolically process it through a network and output the loss

In [3]:
net = X_batch

# add some convolutional layers:
net = tf.layers.conv2d(net, filters=32, kernel_size=3, padding='SAME', activation=tf.nn.relu)
net = tf.layers.conv2d(net, filters=32, kernel_size=3, padding='SAME', activation=tf.nn.relu)
net = tf.layers.max_pooling2d(net, pool_size=2, strides=2)

# add some more if you want:
net = tf.layers.conv2d(net, filters=64, kernel_size=3, padding='SAME', activation=tf.nn.relu)
net = tf.layers.conv2d(net, filters=64, kernel_size=3, padding='SAME', activation=tf.nn.relu)
net = tf.layers.max_pooling2d(net, pool_size=2, strides=2)

# fully connected layers:
net = tf.layers.flatten(net)
net = tf.layers.dense(net, units=512, activation=tf.nn.relu)
net = tf.layers.flatten(net)
net = tf.layers.dense(net, units=10)

logits = net

loss = tf.losses.softmax_cross_entropy(onehot_labels=tf.one_hot(y_batch, depth=10), logits=logits)

# given a loss, create an op that, when run, descends the gradient by one step

In [4]:
train_op = tf.train.GradientDescentOptimizer(learning_rate=.001).minimize(loss)

# start a tensorflow session and use it to initialize all <u>variables</u> and <u>ops</u>

In [5]:
config = tf.ConfigProto(device_count={'GPU': 0})
sess = tf.InteractiveSession(config=config)
sess.run(tf.global_variables_initializer())

# do gradient descent: run a train loop over multiple iterations

In [10]:
sess.run(batch_generator.initializer, feed_dict={X_train_or_test: X_train, y_train_or_test: y_train})
for i in range(1000):
    _, loss_i = sess.run([train_op, loss])
    if i%100 == 0:
        print(loss_i)

0.026054736
0.0230975
0.10829577
0.09332211
0.1412734


# pass through the validation set

In [11]:
sess.run(batch_generator.initializer, feed_dict={X_train_or_test: X_test, y_train_or_test: y_test})
correct = 0
total = 0
for i in range(100):
    prediction, truth = sess.run([logits, y_batch])
    correct += np.sum(prediction.argmax(1)==truth)
    total += len(truth)
acc = correct/total
print(acc)

0.9515625
